# Plan

1. We can narrow down the list of companies by filtering out those with many missing values in the ycharts operating margin dataset.
1. Next, we can use the yfinance python library to check which of the remaining companies have stock prices available in the yfinance dataset.
1. Finally, we can use ycharts to obtain the turnover for the companies that passed both filters.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
ychopmdf = pd.read_excel(
    "data/ycharts_opm/ycharts_operating_margins_dropped.xlsx",
    index_col=0,
    parse_dates=True,
)
# Filter by April, because it's when I compose portfolios
ychopmdf.query("index.dt.month == 4", inplace=True)

ychopmdf.dropna(axis=1, thresh=12, inplace=True) # dropping NaNs
ychopmdf.drop("2009-04-01", inplace=True) # dropping year 2009, to few values

ychopmdf.info()
